In [40]:
import pandas as pd

df = pd.read_csv('Snappfood.csv', encoding='utf-8', on_bad_lines='skip', header=None)


split_columns = df[0].str.split('\t', n=2, expand=True)
split_columns.columns = ['label_id', 'comment','label']
for col in split_columns.columns:
        split_columns[col] = split_columns[col].str.replace('\t', '', regex=False)
        split_columns = split_columns.drop(index=0).reset_index(drop=True)
clean_df = split_columns
clean_df.to_csv('Snappfood_clean.csv', index=False)
      
saved_df = pd.read_csv('Snappfood_clean.csv')
def split_words(comments):
    words = comments.split()
    return [f"'{word}'" for word in words]

saved_df['words'] = saved_df['comment'].apply(split_words)
# print(saved_df)
# print('*' * 30)
# print(len(saved_df))
# print('*' * 30)
all_words = saved_df['words'].explode()
# print(all_words)
# print('*' * 30)
# print(len(all_words))
# print('*' * 30)
drop_duplicated = all_words.drop_duplicates()
print(drop_duplicated)

# print('*' * 30)
# print(len(drop_duplicated))




0             'قیمت'
0              'این'
0              'مدل'
0             'اصلا'
0               'با'
            ...     
69730     'دارم!!!؟'
69733     'عااااشقش'
69741    'تعجبه!!!!'
69741     'دورریختن'
69746      'چربی‌اش'
Name: words, Length: 46431, dtype: object


0          'واقعا'
0            'حیف'
0            'وقت'
0             'که'
0         'بنویسم'
           ...    
69998        'هوا'
69999      'فلفلش'
69999    'خییییلی'
69999        'تند'
69999       'بود.'
Name: words, Length: 1295282, dtype: object
******************************
1295282
******************************
0            'واقعا'
0              'حیف'
0              'وقت'
0               'که'
0           'بنویسم'
            ...     
69981      'بود,فقط'
69981      'مسیرمون'
69984    'تعجبه!!!!'
69984     'دورریختن'
69989      'چربی‌اش'
Name: words, Length: 46926, dtype: object
******************************
46926


Empty DataFrame
Columns: [word]
Index: []
